### 이미지 분류 (Convolutional Neural Network)의 기본적인 코드를
전체적으로 살펴보고 흐름을 이해합니다. 
세부적인 코드 내용은 이후 강의에서 자세히 다룹니다

https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html

* 딥러닝에서는 주로 GPU를 사용하여 학습합니다. (CPU로 학습 시에는 학습 속도가 오래 걸림)
* Colab의 런타임에서 런타임 유형을 GPU로 설정하고 진행하세요.  

### 1. 필요한 라이브러리를 불러옵니다. 
* import 라이브러리명
* from 라이브러리명 import 함수명

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### 2. 데이터셋을 불러옵니다. 

* 여기에서는 Pytorch에서 제공하는 데이터셋을 불러옵니다. 
다른 데이터셋을 불러오는 방법은 이후 강의에서 다룹니다. 

* Train, Test 또는 Train, Validation, Test 로 데이터를 분류하여 학습에 사용하고, 추후 검증에 사용하는 데이터를 나눕니다. 

* Pytorch에서 제공하는 데이터셋 중에 FashionMNIST 를 불러옵니다. 

In [ ]:
# 학습용 데이터셋
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 테스트 데이터셋셋
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



### 3. 데이터셋을 확인합니다

* batch size는 한번에 몇개씩 읽을 것인지를 지정합니다.
* pytorch에서는 이미지를 [N, C, H, W] 로 지정합니다. Color Channel의 위치가 Tensorflow, OpenCV 등과 다릅니다

In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### 4. 모델을 구성합니다. 



In [ ]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 5. 모델 학습 관련한 함수를 지정합니다. 

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### 6. Train 함수를 작성합니다. 

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

### 7. Train 함수를 수정하여 Test 함수를 작성합니다. 

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### 8. 데이터 학습을 진행합니다. 

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302124  [    0/60000]
loss: 2.291876  [ 6400/60000]
loss: 2.278030  [12800/60000]
loss: 2.267909  [19200/60000]
loss: 2.261204  [25600/60000]
loss: 2.223448  [32000/60000]
loss: 2.228845  [38400/60000]
loss: 2.201613  [44800/60000]
loss: 2.203657  [51200/60000]
loss: 2.156635  [57600/60000]
Test Error: 
 Accuracy: 43.2%, Avg loss: 2.162632 

Epoch 2
-------------------------------
loss: 2.169872  [    0/60000]
loss: 2.161865  [ 6400/60000]
loss: 2.110152  [12800/60000]
loss: 2.124492  [19200/60000]
loss: 2.076694  [25600/60000]
loss: 2.011497  [32000/60000]
loss: 2.036125  [38400/60000]
loss: 1.967210  [44800/60000]
loss: 1.975215  [51200/60000]
loss: 1.878802  [57600/60000]
Test Error: 
 Accuracy: 57.8%, Avg loss: 1.895359 

Epoch 3
-------------------------------
loss: 1.919753  [    0/60000]
loss: 1.893505  [ 6400/60000]
loss: 1.785347  [12800/60000]
loss: 1.826197  [19200/60000]
loss: 1.715906  [25600/60000]
loss: 1.659719  [32000/600

### 코딩 이해 필요한 부분
  * import
  * help, dir, 공식 문서 확인 등
  * 기본 문법: for, if, print, data type 등
  * list, dict
    https://wikidocs.net/43
  * 함수 
  * Class
  * 이터레이터: __iter__, __next__
  * 객체 슬라이싱: __getitem__
  * type hint


In [ ]:
dir(test_dataloader)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_index_sampler',
 '_is_protocol',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [12]:
from torchvision.transforms import ToTensor

In [13]:
dir(ToTensor)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']